<a href="https://colab.research.google.com/github/anushadudi/inference_latency_optimization/blob/main/hf_pipeline_bench.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets evaluate accelerate torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.9/484.9 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 59.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 61.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 52.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 39.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 97.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer,AutoModelForSeq2SeqLM
import torch
from huggingface_hub import login
import time
import pandas as pd
from google.colab import userdata

questions = [
    # Coding questions
    "Implement a Python function to compute the Fibonacci numbers.",
    "Write a Rust function that performs binary exponentiation.",
    "What are the differences between Javascript and Python?",
    # Literature
    "Write a story in the style of James Joyce about a trip to the Australian outback in 2083, to see robots in the beautiful desert.",
    "Who does Harry turn into a balloon?",
    "Write a tale about a time-traveling historian who's determined to witness the most significant events in human history.",
    # Math
    "What is the product of 9 and 8?",
    "If a train travels 120 kilometers in 2 hours, what is its average speed?",
    "Think through this step by step. If the sequence a_n is defined by a_1 = 3, a_2 = 5, and a_n = a_(n-1) + a_(n-2) for n > 2, find a_6.",
]


def initiateModel(device):
  model_id = "meta-llama/Meta-Llama-3.1-8B-Instruct"
  login(token=userdata.get('HF_TOKEN'))
  model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.bfloat16)
  model.to(device)
  return model

def initiateTokenizer():
  model_id = "meta-llama/Meta-Llama-3.1-8B-Instruct"
  tokenizer = AutoTokenizer.from_pretrained(model_id)
  tokenizer.add_special_tokens({'pad_token': '[PAD]'})
  return tokenizer


In [ ]:
import torch
from transformers import GenerationConfig
def generate(question, model, tokenizer, device):
  generation_config = GenerationConfig(
    max_new_tokens=200, top_p=0.01, temperature=0.01, do_sample=True, pad_token_id=tokenizer.eos_token_id
  )
  start = time.perf_counter()
  inputs = tokenizer(question, return_tensors="pt").to(device)
  result = model.generate(**inputs, generation_config=generation_config)
  decoded_output = tokenizer.batch_decode(result)[0]
  request_time = time.perf_counter() - start
  response = {'question': question}
  response['tok_count'] = len(result[0])
  response['time'] = request_time
  response['answer'] = decoded_output
  response['tokens_per_second'] = len(result[0]) / request_time
  response['ms_per_seq_output_token'] = request_time * 1000 / len(result[0])
  return response

In [ ]:
def run_benchmark(model, tokenizer, device):
    counter = 1
    responses = []

    for q in questions:
        response = generate(question=q, model=model, tokenizer=tokenizer, device=device)
        if counter >= 2:
            responses.append(response)
        counter += 1

    df = pd.DataFrame(responses)
    df.to_csv('bench-hf.csv', index=False)
    return df

In [ ]:
device = "cuda"
model = initiateModel(device)


config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [ ]:
tokenizer = initiateTokenizer()

In [ ]:
result = run_benchmark(model, tokenizer, device)
result

,question,tok_count,time,answer,tokens_per_second,ms_per_seq_output_token
0,Write a Rust function that performs binary exp...,211,8.398337,<|begin_of_text|>Write a Rust function that pe...,25.124020,39.802547
1,What are the differences between Javascript an...,210,8.421920,<|begin_of_text|>What are the differences betw...,24.934930,40.104383
2,Write a story in the style of James Joyce abou...,231,8.395468,<|begin_of_text|>Write a story in the style of...,27.514844,36.344019
3,Who does Harry turn into a balloon?,209,8.329711,<|begin_of_text|>Who does Harry turn into a ba...,25.090907,39.855076
4,Write a tale about a time-traveling historian ...,224,8.425154,<|begin_of_text|>Write a tale about a time-tra...,26.587052,37.612293
5,What is the product of 9 and 8?,212,8.404619,<|begin_of_text|>What is the product of 9 and ...,25.224225,39.644430
6,"If a train travels 120 kilometers in 2 hours, ...",219,8.399660,<|begin_of_text|>If a train travels 120 kilome...,26.072484,38.354612
7,Think through this step by step. If the sequen...,258,8.424978,<|begin_of_text|>Think through this step by st...,30.623226,32.654953
